# Exercice 02 - Introduction au Data Lake

## Objectifs
- Comprendre ce qu'est un Data Lake
- Différencier Data Lake et Data Warehouse
- Découvrir l'architecture Bronze / Silver / Gold
- Comprendre le rôle de MinIO dans notre infrastructure

---

## 1️⃣ Qu'est-ce qu'un Data Lake ?

Un **Data Lake** (lac de données) est un système de stockage qui permet de conserver des données **brutes** dans leur format d'origine.

### Analogie du lac

```
┌─────────────────────────────────────────────────────────────────────┐
│                         🏔️ SOURCES DE DONNÉES                       │
│                                                                     │
│   📊 CSV    📝 JSON    🖼️ Images    📹 Vidéos    🔢 Logs           │
│      │         │          │           │           │                │
│      └─────────┴──────────┴───────────┴───────────┘                │
│                           │                                         │
│                           ▼                                         │
│              ════════════════════════════                           │
│           ══                              ══                        │
│        ══          🌊 DATA LAKE 🌊          ══                     │
│       ║                                      ║                      │
│       ║   Stockage de TOUTES les données     ║                      │
│       ║   dans leur format ORIGINAL          ║                      │
│       ║                                      ║                      │
│        ══                                  ══                       │
│           ══                            ══                          │
│              ════════════════════════════                           │
│                           │                                         │
│                           ▼                                         │
│                    Traitement & Analyse                             │
└─────────────────────────────────────────────────────────────────────┘
```

### Caractéristiques
- Stocke les données **brutes** (pas de transformation à l'entrée)
- Accepte **tous les formats** (CSV, JSON, Parquet, images...)
- **Schéma à la lecture** (schema-on-read)
- Coût de stockage **faible**

## 2️⃣ Data Lake vs Data Warehouse

```
┌──────────────────────────────────┬──────────────────────────────────┐
│          DATA LAKE               │         DATA WAREHOUSE           │
├──────────────────────────────────┼──────────────────────────────────┤
│                                  │                                  │
│  📥 Données BRUTES               │  📊 Données STRUCTURÉES          │
│                                  │                                  │
│  ┌─────────────────────────┐     │  ┌─────────────────────────┐    │
│  │ {"name": "John", ...}  │     │  │ ID │ NOM  │ PRENOM │...│    │
│  │ user,age,city          │     │  │ 1  │ Doe  │ John   │   │    │
│  │ <xml>...</xml>         │     │  │ 2  │ Smith│ Jane   │   │    │
│  │ 🖼️ image.jpg            │     │  └─────────────────────────┘    │
│  └─────────────────────────┘     │                                  │
│                                  │                                  │
│  ✅ Tous formats acceptés        │  ✅ Données prêtes à l'emploi    │
│  ✅ Stockage peu coûteux         │  ✅ Requêtes SQL rapides         │
│  ✅ Flexible                     │  ✅ Qualité garantie             │
│                                  │                                  │
│  ❌ Nécessite transformation     │  ❌ Schéma rigide                │
│  ❌ Peut devenir un "data swamp" │  ❌ Coût élevé                   │
│                                  │                                  │
└──────────────────────────────────┴──────────────────────────────────┘
```

### En résumé
- **Data Lake** = Stockage brut, flexible, pour exploration
- **Data Warehouse** = Données structurées, pour reporting

## 3️⃣ Architecture Bronze / Silver / Gold

Pour éviter que le Data Lake devienne un "marécage" (data swamp), on organise les données en **3 couches** :

```
┌─────────────────────────────────────────────────────────────────────┐
│                    ARCHITECTURE MEDALLION                           │
│                  (Bronze / Silver / Gold)                           │
└─────────────────────────────────────────────────────────────────────┘

   SOURCES                BRONZE              SILVER              GOLD
   ───────                ──────              ──────              ────
                                                                      
 ┌─────────┐         ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
 │   API   │────────▶│             │    │             │    │             │
 └─────────┘         │             │    │             │    │             │
                     │   DONNÉES   │    │   DONNÉES   │    │   DONNÉES   │
 ┌─────────┐         │   BRUTES    │───▶│  NETTOYÉES  │───▶│  AGRÉGÉES   │
 │   CSV   │────────▶│             │    │             │    │             │
 └─────────┘         │  (raw)      │    │ (cleaned)   │    │ (business)  │
                     │             │    │             │    │             │
 ┌─────────┐         └─────────────┘    └─────────────┘    └─────────────┘
 │   DB    │────────▶      │                  │                  │
 └─────────┘               │                  │                  │
                           ▼                  ▼                  ▼
                      
                    📦 Stockage         🔧 Qualité         📊 Analytics
                    rapide              données            & Reporting
```

### 🥉 Bronze (Brut)
- Données **exactement comme reçues**
- Aucune transformation
- Format original conservé
- Sert de "sauvegarde" des données sources

### 🥈 Silver (Nettoyé)
- Données **nettoyées et validées**
- Types corrigés
- Valeurs nulles traitées
- Doublons supprimés
- Format standardisé (souvent Parquet)

### 🥇 Gold (Agrégé)
- Données **prêtes pour l'analyse**
- Agrégations métier
- Indicateurs calculés
- Optimisé pour les requêtes

## 4️⃣ Notre infrastructure locale

Dans notre environnement Docker, voici comment les composants s'organisent :

```
┌─────────────────────────────────────────────────────────────────────┐
│                      NOTRE DATA LAKE LOCAL                          │
└─────────────────────────────────────────────────────────────────────┘

    ┌───────────────┐     ┌───────────────┐     ┌───────────────┐
    │  POSTGRESQL   │     │    KAFKA      │     │   INTERNET    │
    │   (Northwind) │     │  (streaming)  │     │    (APIs)     │
    └───────┬───────┘     └───────┬───────┘     └───────┬───────┘
            │                     │                     │
            └─────────────────────┴─────────────────────┘
                                  │
                                  ▼
                    ┌─────────────────────────┐
                    │      SPARK (PySpark)    │
                    │   Moteur de traitement  │
                    └───────────┬─────────────┘
                                │
                                ▼
    ┌─────────────────────────────────────────────────────────────┐
    │                         MINIO                                │
    │                   (Data Lake S3)                             │
    │                                                              │
    │   ┌─────────────┐  ┌─────────────┐  ┌─────────────┐         │
    │   │   BRONZE    │  │   SILVER    │  │    GOLD     │         │
    │   │             │  │             │  │             │         │
    │   │ s3://bronze │  │ s3://silver │  │ s3://gold   │         │
    │   │             │  │             │  │             │         │
    │   └─────────────┘  └─────────────┘  └─────────────┘         │
    │                                                              │
    └─────────────────────────────────────────────────────────────┘
                                  │
                                  ▼
                    ┌─────────────────────────┐
                    │      JUPYTERLAB         │
                    │   (vous êtes ici!)      │
                    └─────────────────────────┘
```

## 5️⃣ Qu'est-ce que MinIO ?

**MinIO** est un serveur de stockage objet compatible avec **Amazon S3**.

### Pourquoi S3 ?
- Standard de l'industrie pour le stockage Big Data
- Tous les outils Big Data savent lire/écrire en S3
- Scalable à l'infini
- Coût faible

### MinIO = S3 en local
- Même API que S3
- Parfait pour le développement
- Gratuit et open source

### Accès à MinIO
- **Console web** : http://localhost:9001
- **API S3** : http://localhost:9000
- **Credentials** : minioadmin / minioadmin123

## 6️⃣ Démonstration : Connexion à MinIO

In [1]:
pip install minio

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Installation du client MinIO si nécessaire
# !pip install minio

from minio import Minio

# Connexion à MinIO
client = Minio(
    "minio:9000",                    # Adresse du serveur (nom du container Docker)
    access_key="minioadmin",          # Identifiant
    secret_key="minioadmin123",       # Mot de passe
    secure=False                      # Pas de HTTPS en local
)

print("✅ Connexion à MinIO réussie !")

✅ Connexion à MinIO réussie !


In [3]:
# Lister les buckets existants
buckets = client.list_buckets()

print("Buckets existants :")
if buckets:
    for bucket in buckets:
        print(f"  - {bucket.name}")
else:
    print("  (aucun bucket pour le moment)")

Buckets existants :
  - bronze
  - gold
  - silver


In [4]:
# Créer les buckets Bronze, Silver, Gold
buckets_a_creer = ["bronze", "silver", "gold"]

for bucket_name in buckets_a_creer:
    # Vérifier si le bucket existe déjà
    if not client.bucket_exists(bucket_name):
        client.make_bucket(bucket_name)
        print(f"✅ Bucket '{bucket_name}' créé")
    else:
        print(f"ℹ️  Bucket '{bucket_name}' existe déjà")

ℹ️  Bucket 'bronze' existe déjà
ℹ️  Bucket 'silver' existe déjà
ℹ️  Bucket 'gold' existe déjà


In [5]:
# Vérification : lister à nouveau les buckets
print("\nBuckets après création :")
for bucket in client.list_buckets():
    print(f"  🪣 {bucket.name}")


Buckets après création :
  🪣 bronze
  🪣 gold
  🪣 silver


## 7️⃣ Exemple : Stocker un fichier dans Bronze

In [6]:
import json
from io import BytesIO

# Données exemple
donnees_exemple = {
    "source": "demonstration",
    "date": "2025-01-12",
    "message": "Premier fichier dans le Data Lake !"
}

# Convertir en JSON
json_data = json.dumps(donnees_exemple, indent=2)
json_bytes = json_data.encode('utf-8')

# Uploader vers Bronze
client.put_object(
    bucket_name="bronze",
    object_name="demo/premier_fichier.json",
    data=BytesIO(json_bytes),
    length=len(json_bytes),
    content_type="application/json"
)

print("✅ Fichier uploadé dans bronze/demo/premier_fichier.json")

✅ Fichier uploadé dans bronze/demo/premier_fichier.json


In [7]:
# Lire le fichier depuis Bronze
response = client.get_object("bronze", "demo/premier_fichier.json")
contenu = response.read().decode('utf-8')
response.close()
response.release_conn()

print("Contenu du fichier :")
print(contenu)

Contenu du fichier :
{
  "source": "demonstration",
  "date": "2025-01-12",
  "message": "Premier fichier dans le Data Lake !"
}


---

## Exercice

**Objectif** : Manipuler MinIO et comprendre l'organisation du Data Lake

**Consigne** :
1. Créez un dictionnaire Python avec vos informations (prénom, ville, hobby)
2. Convertissez-le en JSON
3. Uploadez-le dans le bucket `bronze` avec le chemin `exercice/mon_profil.json`
4. Relisez le fichier pour vérifier

À vous de jouer

In [8]:
import json
from io import BytesIO

# TODO: Créez votre dictionnaire
mon_profil = {
    "prenom": "Gills",      # Votre prénom
    "ville": "Lille",       # Votre ville
    "hobby": "Manger"        # Votre hobby
}

# TODO: Convertissez en JSON et uploadez vers bronze/exercice/mon_profil.json
json_data = json.dumps(mon_profil, indent=2)
json_bytes = json_data.encode('utf-8')

client.put_object(
    bucket_name="bronze",
    object_name="exercice/mon_profil.json",
    data=BytesIO(json_bytes),
    length=len(json_bytes),
    content_type="application/json"
)
# Indice : utilisez le même code que l'exemple précédent


ObjectWriteResult(bucket_name='bronze', object_name='exercice/mon_profil.json', version_id=None, etag='5ad08e92c26512cdfc36b81f806a3b8a', http_headers=HTTPHeaderDict({'Accept-Ranges': 'bytes', 'Content-Length': '0', 'ETag': '"5ad08e92c26512cdfc36b81f806a3b8a"', 'Server': 'MinIO', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'Vary': 'Origin, Accept-Encoding', 'X-Amz-Id-2': 'dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8', 'X-Amz-Request-Id': '188A96EF9BF4060D', 'X-Content-Type-Options': 'nosniff', 'X-Ratelimit-Limit': '5623', 'X-Ratelimit-Remaining': '5623', 'X-Xss-Protection': '1; mode=block', 'Date': 'Wed, 14 Jan 2026 11:55:59 GMT'}), last_modified=None, location=None)

In [9]:
client.remove_object(
    bucket_name="bronze",
    object_name="demo/mon_profil.json"
)

In [10]:
# TODO: Relisez le fichier pour vérifier
response = client.get_object("bronze", "demo/mon_profil.json")
contenu = response.read().decode('utf-8')
response.close()
response.release_conn()

print("Contenu du fichier :")
print(contenu)

S3Error: S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /bronze/demo/mon_profil.json, request_id: 188A96F0435E9965, host_id: dd9025bab4ad464b049177c95eb6ebf374d3b3fd1af9251148b658df7ac2e3e8, bucket_name: bronze, object_name: demo/mon_profil.json

In [11]:
# TODO: Relisez le fichier pour vérifier
response = client.get_object("bronze", "exercice/mon_profil.json")
contenu = response.read().decode('utf-8')
response.close()
response.release_conn()

print("Contenu du fichier :")
print(contenu)

Contenu du fichier :
{
  "prenom": "Gills",
  "ville": "Lille",
  "hobby": "Manger"
}


---

## ✅ Résumé

Dans ce notebook, vous avez appris :

- Un **Data Lake** stocke des données brutes de tout format
- Différence entre **Data Lake** (flexible) et **Data Warehouse** (structuré)
- L'architecture **Bronze / Silver / Gold** organise les données
- **MinIO** est notre Data Lake local compatible S3
- Comment **créer des buckets** et **stocker des fichiers**

### Prochaine étape
Dans le prochain notebook, nous découvrirons **Apache Spark** et comment créer notre première **SparkSession**.